## A machine learning algorithm for classification of dermoscopic images

Dermoscopic imaging is a non-invasive imaging technique that allows for direct microscopic examination of the diagnostic features not seen by the naked eye in pigmented skin lesions[1]. Identification and retrieval of dermoscopic images is a task that is fundamental for the population of the Archive of the International Skin Imaging Collaboration (ISIC Archive)[2,3]. As the dermoscopic images of interest have no tags or other metadata indicating that they are dermoscopic in the database from which they are retrieved, identifying and sorting these images can become a very labor-intensive process. As a means toward alleviating the burden associated with sorting and labelling tens of thousands of images, an algorithm for automating said task is outlined here.

In [10]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url='http://github.com/aadikalloo/ND_CNN/raw/master/derm_vs_nderm_2.png')

This system is composed of three main parts:
    1. Reading images, pre-processing, and saving as an array
        a. Extract color histogram data
    2. Training a convolutional neural network to learn features of processed images
    3. Training a random forest classifier to classify images based on color histogram data, using neural network    output classification as an input feature

#### Required Packages

In [ ]:
#import packages:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
import math
import cv2
import sklearn
from sklearn.metrics import auc, roc_curve
import joblib
import multiprocessing
import time
from imutils import paths
import imutils
import os
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.regularizers import l2, activity_l2
from sklearn.cross_validation import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

### Part 1: Preprocessing

In [ ]:
def get_file_list(csv_path):
	files_df = pd.read_csv(csv_path) #read csv with pandas function
	onlyfiles = files_df['Filename'] #select only filename column
	status_df = files_df['Status']
	return onlyfiles, status_df #return list of image files

def extract_color_histogram(image, bins=(8*3, 8*3, 8*3)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	# otherwise, perform "in place" normalization in OpenCV 3 (I
	else:
		cv2.normalize(hist, hist)
	# return the flattened histogram as the feature vector
	return hist.flatten()

def readImage(num, file, image_path, wantStatus, status_df=[]):
	file = image_path + file
	img0 = cv2.imread(file) #read image
	if num % 100 == 0:
		print(num) #basic progress indicator
	color_hist = extract_color_histogram(img0)
	img = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY) #convert image to grayscale
	kernel = np.ones((2,2), np.float32)/4 #create 2x2 kernel for smoothing
	img = cv2.filter2D(img, -1, kernel) #smooth image
	laplacian = cv2.Laplacian(img, cv2.CV_8U) #use laplacian filter ("edge detector") to find edges and lines
	laplacian = cv2.filter2D(laplacian, -1, kernel) #smooth image
	laplacian = cv2.GaussianBlur(laplacian, (3, 3), 0) #blur image
	laplacian = (255 - laplacian) #invert color/gray values
	laplacian = cv2.GaussianBlur(laplacian, (7, 7), 0) #blur image
	ret, laplacian = cv2.threshold(laplacian, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) #use Otsu adaptive thresholding to extract scale bar
	laplacian = cv2.resize(laplacian, (200, 200)) #convert to 500x500 and return
	status = status_df[num]
	return (laplacian, file, color_hist, status)
	
if __name__ == '__main__':
	wantStatus = True
	image_path = 'image_folder' #location of images
	csv_path = 'csv_path' #location of csv with filename and dermoscopy status; columns: Filename, Status
	list_of_images, status_df = get_file_list(csv_path)
	time1 = time.time()
	num_cores = multiprocessing.cpu_count()
	result_array = joblib.Parallel(n_jobs=num_cores)(joblib.delayed(readImage)(i, image, image_path, wantStatus, status_df) for i, image in enumerate(list_of_images))
	time2 = time.time()
	print('read function took %0.3f s' % ((time2-time1)*1))
	print("Saving...")
	np.save('array_save_path', result_array)
	print("Model saved.")


The above code is used to resize, and apply convolutional filters, and append images to a 3D array. This results in images with dimensions 200px by 200px. The filters are used to "pull out" the scale bar. These scale bars are used as the primary feature recognized by the CNN when learning to classify dermoscopic images. The result of this step is shown below:

In [11]:
Image(url='https://github.com/aadikalloo/ND_CNN/raw/master/derm_processed_2.png')

### Part 2: Training the neural network

In [ ]:
def reshape_image_array(image_array):
    image_array = image_array[..., 0]
    image_array = np.dstack(image_array)
    image_array = np.rollaxis(image_array, -1)
    image_array = image_array[:, np.newaxis, :, :]
    return image_array

def load_data(csv_path, image_path):
     #location of csv with filename and dermoscopy status; columns: Filename, Status
    image_array_with_filenames = np.load(image_path)
    filename_array = image_array_with_filenames[..., 1]
    status_array = image_array_with_filenames[...,3]
    allimages = reshape_image_array(image_array_with_filenames)
    del image_array_with_filenames
    random.seed(1149)
    shuffled_indices = np.random.permutation(np.arange(len(filename_array)))
    shuffled_images = allimages[shuffled_indices]
    shuffled_df = filename_array[shuffled_indices]
    shuffled_status = status_array[shuffled_indices]
    shuffled_filedf = pd.DataFrame(shuffled_df, columns = ['Filename'])
    shuffled_filedf['Status'] = shuffled_status
    return shuffled_images, shuffled_filedf

def split_datasets(shuffled_images, shuffled_df, train_proportion):
    X_train = shuffled_images[0:math.floor(train_proportion*len(shuffled_df))]
    X_test = shuffled_images[(math.floor(train_proportion*len(shuffled_df))+1):len(shuffled_df)]
    df_train = shuffled_df[0:math.floor(train_proportion*len(shuffled_df))]
    df_test = shuffled_df[(math.floor(train_proportion*len(shuffled_df))+1):len(shuffled_df)]
    y_train = df_train['Status'].values
    y_test = df_test['Status'].values
    Y_train = np_utils.to_categorical(y_train, 2)
    Y_test = np_utils.to_categorical(y_test, 2)
    del shuffled_images, shuffled_df
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    return X_train, X_test, Y_train, Y_test, df_train, df_test    

def create_model(num_conv_filters_layer1, num_conv_kernel_rows, num_conv_kernel_cols, num_conv_filters_layer2):
    model = Sequential()
    act = 'relu' #relu
    model.add(Convolution2D(num_conv_filters_layer1, num_conv_kernel_rows, num_conv_kernel_cols, border_mode='same', input_shape=(img_channels, img_rows, img_cols)))
    model.add(Activation(act))
    model.add(Convolution2D(num_conv_filters_layer1, num_conv_kernel_rows, num_conv_kernel_cols))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(num_conv_filters_layer2, num_conv_kernel_rows, num_conv_kernel_cols, border_mode='same'))
    model.add(Activation(act))
    model.add(Convolution2D(num_conv_filters_layer2, num_conv_kernel_rows, num_conv_kernel_cols))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512)) #model.add(Dense(512))
    model.add(Dropout(0.8)) #0.5
    model.add(Activation(act))
    model.add(Dense(128)) #model.add(Dense(512)) #added
    model.add(Dropout(0.5)) #0.5 #added
    model.add(Activation(act)) #added
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    return model

if __name__ == '__main__':
    num_conv_filters_layer1 = 128
    num_conv_filters_layer2 = 32
    num_conv_kernel_rows, num_conv_kernel_cols = 3, 3
    batch_size = 48 #originally 32
    nb_classes = 2
    nb_epoch = 100
    data_augmentation = True
    test_proportion = 0.25
    train_proportion = 1 - test_proportion
    img_rows, img_cols = 200, 200
    img_channels = 1 #3
    learning_rate = 0.001
    n_folds = 10
    image_path = 'array_save_path'
    csv_path = 'csv_path'
    print("Loading data...")
    Shuffled_images, Shuffled_filedf = load_data(csv_path, image_path)
    print("Data loaded.")
    train_data, test_data, train_labels, test_labels, train_filenames, test_filenames = split_datasets(Shuffled_images, Shuffled_filedf, train_proportion)
    print("Datasets split.")
    model = create_model(32, 3, 3, 32)
    sgd = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print("Model created.")
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(train_data, train_labels,
                  batch_size=batch_size,
                  nb_epoch=nb_epoch,
                  validation_data=(test_data, test_labels),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        datagen.fit(train_data)

        # fit the model on the batches generated by datagen.flow()
        model.fit_generator(datagen.flow(train_data, train_labels,
                            batch_size=batch_size),
                            samples_per_epoch=25000,
                            nb_epoch=nb_epoch,
                            validation_data=(test_data, test_labels))
    model.save('save_model_path')


This network reads the image array of 200x200px images and trains using real-time data augmentation. GPU-acceleration was enabled to speed up training (NVIDIA Titan X). Before augmentation, 77000 images (inclusive of dermoscopic and non-dermoscopic) were used to train the network over 100 epochs (25000 images per epoch). A diagram of the network architecture is shown below:

In [12]:
Image(url='https://github.com/aadikalloo/ND_CNN/raw/master/nn7_plot.png')

As summarized here, this architecture allowed the network to achieve validation accuracies (accuracy on data not used for training) of ~93.5% at the end of training: 
    
    Epoch 95/100
    25037/25000 - 118s - loss: 0.1511 - acc: 0.9436 - val_loss: 0.1738 - val_acc: 0.9356

    Epoch 96/100
    25008/25000 - 117s - loss: 0.1469 - acc: 0.9452 - val_loss: 0.1723 - val_acc: 0.9360

    Epoch 97/100
    25008/25000 - 117s - loss: 0.1498 - acc: 0.9439 - val_loss: 0.1709 - val_acc: 0.9362

    Epoch 98/100
    25037/25000 - 118s - loss: 0.1445 - acc: 0.9483 - val_loss: 0.1828 - val_acc: 0.9289

    Epoch 99/100
    25008/25000 - 117s - loss: 0.1458 - acc: 0.9457 - val_loss: 0.1771 - val_acc: 0.9348

    Epoch 100/100
    25037/25000 - 117s - loss: 0.1524 - acc: 0.9453 - val_loss: 0.1816 - val_acc: 0.9305

While an accuracy of 93% was a major stride toward automating image classification, it was hypothesized that the use of an additional classifier could increase validation accuracy.

### Part 3: Random Forest classifier using nn output + color histograms

In [ ]:
def load_data(results, image_array_path):
	result_array = pd.read_csv(results)
	result_array = result_array.iloc[:,0]
	image_array_with_filenames = np.load(image_array_path) #location of images
	return result_array, image_array_with_filenames

def form_hist_2D_array(image_array_with_filenames, result_array):
	hist_data = image_array_with_filenames[..., 2]
	hist_data = np.dstack(hist_data)
	hist_data = hist_data.reshape(hist_data.shape[1:])
	hist_data = np.rollaxis(hist_data, -1)
	status_array = image_array_with_filenames[...,3]
	status_array = status_array.tolist()
	status_array = np.asarray(status_array)
	hist_data = np.insert(hist_data, 0, values=result_array, axis=1)
	return hist_data, status_array

def output_metric_results(crosstabresults, score):
	TN = crosstabresults[0][0]
	FN = crosstabresults[0][1]
	FP = crosstabresults[1][0]
	TP = crosstabresults[1][1]
	print(crosstabresults)
	Sensitivity = TP/(TP+FN)
	Specificity = TN/(TN+FP)
	PPV = TP/(TP+FP)
	NPV = TN/(TN+FN)
	print("Accuracy: ", score)
	print("Sensitivity: ", Sensitivity)
	print("Specificity: ", Specificity)
	print("PPV: ", PPV)
	print("NPV: ", NPV)

def train_RF(hist_data, status_array, train_prop):
	arr_len = len(hist_data)
	train_len = math.floor(arr_len*train_prop)
	model = RandomForestClassifier(n_estimators=100, max_features=None, n_jobs=-1, verbose=True)
	model.fit(hist_data[0:train_len], status_array[0:train_len])
	return model, arr_len, train_len

if __name__ == '__main__':
	train_prop = 0.75
	results = 'nn_predictions.csv' #predictions
	image_array_path = 'array_path'
	pickle_loc = 'rf_save_path'

	result_array, image_array_with_filenames = load_data(results, image_array_path)
	hist_data, status_array = form_hist_2D_array(image_array_with_filenames, result_array)
	model, arr_len, train_len = train_RF(hist_data, status_array, train_prop)
	preds = model.predict(hist_data[(train_len+1):arr_len])
	score = accuracy_score(status_array[(train_len+1):arr_len], preds)
	joblib.dump(model, pickle_loc) 
	crosstabresults = pd.crosstab(status_array[(train_len+1):arr_len], preds, rownames=['actual'], colnames=['preds:'])
	output_metric_results(crosstabresults, score)


#### Classification Metrics

    preds:     0     1
    actual            
    0       4022    88
    1        237  2673
    
    Accuracy:  0.953703703704
    Sensitivity:  0.918556701031
    Specificity:  0.978588807786
    PPV:  0.96812749004
    NPV:  0.944353134539

#### ROC Curve

In [13]:
Image(url='https://github.com/aadikalloo/ND_CNN/raw/master/Versions/nn4.png')

Over multiple testing phases, this classifier has consistently achieved accuracies >95%. 

In [ ]:
References:
    1. Vestergaard, M. E., Macaskill, P. H. P. M., Holt, P. E., & Menzies, S. W. (2008). Dermoscopy compared with naked eye examination for the diagnosis of primary melanoma: a meta‐analysis of studies performed in a clinical setting. British Journal of Dermatology, 159(3), 669-676.
    2. Gutman, D., Codella, N. C., Celebi, E., Helba, B., Marchetti, M., Mishra, N., & Halpern, A. (2016). Skin Lesion Analysis toward Melanoma Detection: A Challenge at the International Symposium on Biomedical Imaging (ISBI) 2016, hosted by the International Skin Imaging Collaboration (ISIC). arXiv preprint arXiv:1605.01397.
    3. International Skin Imaging Collaboration. www.isic-archive.com
    4. 